In [1]:
import torch
import torchaudio
import torch.utils.data as data
import torchvision.transforms as transforms
import numpy as np
from pytorch_nsynth.nsynth import NSynth, SignalTransformation
import sys
import librosa.display
import json
import matplotlib.pyplot as plt
torch.cuda.is_available()

/home/alexis/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

In [3]:
dataset = NSynth(
        "nsynth-valid",
        transform=toFloat,
        blacklist_pattern=["string"],  # blacklist string instrument
        categorical_field_list=["instrument_family", "instrument_source"])

In [4]:
loader = data.DataLoader(dataset, batch_size=32, shuffle=True)

# Coger el archivo de audio con source 0 y clasificar por familias


### Families
0 - Bass

1 - Brass

2 - Flute

3 - Guitar

4 - Keyboard

5 - Mallet

6 - Organ

7 - Reed

8 - String

9 - Synth_lead

10 - Vocal

In [5]:
# print(dataset.__getitem__(0))
# print(dataset.__getitem__(0)[3]["note_str"])
# print(dataset.__getitem__(0)[3]["instrument_source"])
print(dataset.getAudioCharacteristics(0))

{'fichero': 'bass_synthetic_033-087-050.wav', 'familia': 2}


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# audioCharacteristics=dataset.getAudioCharacteristics(0)
# signalTransformation = SignalTransformation(fichero=audioCharacteristics["fichero"],label=audioCharacteristics["familia"])
# plot_specgram(signalTransformation.generarSpectrograma(),16000,title=audioCharacteristics["fichero"])

with open("nsynth-test/examples.json", "r") as f:
    datos = json.loads(f.read())

leyenda = {}
contador = 0
for dato in datos:
        waveform, sample_rate = torchaudio.load("./nsynth-test/audio/"+dato+".wav")
        waveform=waveform.to(device)
        waveform = SignalTransformation.generarSpectrogramaFromSignal(waveform)
        shape = waveform.shape
        waveform = torchaudio.transforms.AmplitudeToDB()(waveform) 
        waveform=waveform.cpu().data.numpy()
        librosa.display.specshow(waveform[0])
        plt.colorbar()
        plt.savefig("./nsynth-test/spectrograms/"+dato+".png", bbox_inches='tight')
        plt.close()
#         leyenda[dato] = datos[dato]["instrument_family"]


# Ejemplo hecho con Librosa

In [ ]:

# with open("nsynth-test/examples.json", "r") as f:
#     datos = json.loads(f.read())

# leyenda = {}
# contador = 0
# for dato in datos:
#     if(contador>200):
#         break;
#     else:
#         waveform, sample_rate = librosa.load("./nsynth-test/audio/"+dato+".wav")
# #         waveform=waveform.to(device)
#         shape = waveform.shape
# #         waveform = SignalTransformation.generarSpectrogramaFromSignal(waveform)
#         waveform=librosa.feature.melspectrogram(y=waveform, sr=sample_rate)
# #         plot_waveform(waveform, sample_rate,shape)
# #         leyenda[dato] = datos[dato]["instrument_family"]
#         waveform=np.abs(waveform)
#         waveform=librosa.power_to_db(waveform,ref=np.max)
#         librosa.display.specshow(waveform)
#         plt.savefig("./nsynth-test/spectrograms/"+dato+".png", bbox_inches='tight')
#         plt.close()
#         contador+=1
        